<a href="https://colab.research.google.com/github/diyamanapetty/PRODIGY_ML_03/blob/main/Task_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_folder = '/content/drive/My Drive/train'
test_folder = '/content/drive/My Drive/test'

In [4]:
def load_images_from_folder(folder, label, image_size=(64, 64)):
    images, labels = [], []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img_resized = cv2.resize(img, image_size)
            images.append(img_resized.flatten())
            labels.append(label)
    return images, labels

In [5]:
def load_data(data_folder, image_size=(64, 64)):
    cats_folder = os.path.join(data_folder, 'cats')
    dogs_folder = os.path.join(data_folder, 'dogs')
    cat_images, cat_labels = load_images_from_folder(cats_folder, 0, image_size)
    dog_images, dog_labels = load_images_from_folder(dogs_folder, 1, image_size)
    return np.array(cat_images + dog_images), np.array(cat_labels + dog_labels)

In [6]:
X_train, y_train = load_data(train_folder)
X_test, y_test = load_data(test_folder)

In [14]:
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 0.01, 0.1, 1],
    'kernel': ['linear', 'rbf']
}

In [17]:
grid_search = GridSearchCV(SVC(), param_grid, scoring='accuracy', cv=3, verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


GridSearchCV(cv=3, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'gamma': ['scale', 0.01, 0.1, 1],
                         'kernel': ['linear', 'rbf']},
             scoring='accuracy', verbose=2)

In [18]:
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

In [19]:
print("Best Parameters:", best_params)

Best Parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}


In [20]:
y_pred = best_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6428571428571429
Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.66      0.65        70
           1       0.65      0.63      0.64        70

    accuracy                           0.64       140
   macro avg       0.64      0.64      0.64       140
weighted avg       0.64      0.64      0.64       140



In [21]:
def preprocess_image(image_path, image_size=(64, 64)):
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, image_size)
    return img_resized.flatten()

In [22]:
test_image_path = '/content/drive/My Drive/test/cats/cat_147.jpg'
test_image_data = preprocess_image(test_image_path)
prediction = best_model.predict([test_image_data])

In [23]:
print(f"The image is classified as: {'Cat' if prediction[0] == 0 else 'Dog'}")

The image is classified as: Cat
